In [1]:
import praw
import numpy as np
import pandas as pd
from collections import defaultdict
from time import strftime, gmtime
import re
from tqdm import tqdm
import pprint
import os
import datetime as dt
import psaw
from psaw import PushshiftAPI

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option("max_rows", None)

In [3]:
def reddit_info(subr):
    print(subr.id)
    pprint.pprint(vars(subr))

In [4]:
# settings.py
from dotenv import load_dotenv
load_dotenv()

# OR, explicitly providing path to '.env'
from pathlib import Path  # python3 only
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

True

True

In [5]:
client_id = os.getenv("r_client_id")
client_secret = os.getenv("r_client_secret")
user_agent = os.getenv("r_user_agent")
username = os.getenv("r_username")
password = os.getenv("r_password")

In [6]:
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent,
                     username=username,
                     password=password)

In [107]:
df_sub = pd.read_csv('data\df_sub.csv', index_col=0)
df_sub.head()

,name,sub_name,sub_pub_desc,subscribers
0,Grand Theft Auto V,GTAV,"Grand Theft Auto V, also known as GTA 5 or GTA...",256651.0
1,Portal 2,Portal,Welcome to Aperture Laboratories. The subreddi...,77681.0
2,The Witcher 3: Wild Hunt,thewitcher3,For everything Witcher related.,67841.0
3,The Elder Scrolls V: Skyrim,skyrim,NaN,891740.0
4,Tomb Raider (2013),TombRaider,A subreddit for people interested in Lara Crof...,30280.0


In [8]:
api = PushshiftAPI(reddit)

In [159]:
result = api.search_comments(author='Bigfootslides', aggs='id')

In [161]:
result = api.redditor_subreddit_activity(user)

In [162]:
type(result)

dict

In [139]:
def gamesub4user(user):
    result = api.redditor_subreddit_activity(user)
    game_sub = set(result['comment'].keys()).intersection(set(df_sub.sub_name))
    for subr in result
    
    return list(game_sub)

In [294]:
def user_info(sub, user_set, limit=500):
    user_dict = defaultdict(list)
    starttime = int(dt.datetime(2020, 4, 1).timestamp())
    endtime = int(dt.datetime(2020, 5, 1).timestamp())
    bot_list = ['AutoModerator', 'VredditDownloader']
    comments = api.search_comments(after=starttime, before=endtime, limit=limit, subreddit=sub)
    for comment in comments:
        user = comment.author
        if (user not in user_set) and (user not in bot_list) and user:
            #print('  Getting user {}. Time: {}.'.format(user, strftime("%H:%M:%S", gmtime())))
            user_set.add(user)
            user_dict['username'].append(user.name)
            user_dict['subreddit'].append(sub)
    
    return pd.DataFrame(user_dict), user_set

In [313]:
def user_sub_comment(df, sub_list, limit=100):
    comment_dict = defaultdict(list)
    l = len(df)
    for i in tqdm(range(l)):
        #print('-Running user_sub for user {}\'s comment. Time: {}.'.format(user, strftime("%H:%M:%S", gmtime())))
        user = df.username[i]
        user_game_sub = set()
        user_comment_dict = defaultdict(list)
        #print(sub_list)
        user_comments = api.search_comments(author=user, subreddit=sub_list, limit=limit)
        for user_comment in user_comments:
            subr = user_comment.subreddit.display_name
            if subr in sub_list:
                user_game_sub.add(subr)
                user_comment_dict['user'].append(user)
                user_comment_dict['comment'].append(user_comment.body)
                user_comment_dict['subreddit'].append(subr)
                user_comment_dict['score'].append(user_comment.score)
                user_comment_dict['time'].append(user_comment.created_utc)
                
        if len(user_game_sub) >= 3:
            for key in user_comment_dict.keys():
                comment_dict[key] += user_comment_dict[key]
    
    return pd.DataFrame(comment_dict)

In [255]:
'''def user_comment(df, limit=None):
    comment_dict = defaultdict(list)
    l = len(df.username)
    for i in tqdm(range(l)):
        user = df.loc[i,'username']
        user_game_sub = df.loc[i,'subreddit']
        for subr in user_game_sub:
            #print('-Running user_comment, getting user {}\'s comments on subreddit {}. Time: {}.'.format(user, subr, strftime("%H:%M:%S", gmtime())))
            user_comments = api.search_comments(author=user, subreddit=subr, limit=limit)
            for user_comment in user_comments:
                comment_dict['user'].append(user)
                comment_dict['comment'].append(user_comment.body)
                comment_dict['subreddit'].append(subr)
                comment_dict['score'].append(user_comment.score)
                comment_dict['time'].append(user_comment.created_utc)
    return pd.DataFrame(comment_dict)'''

In [316]:
def get_comment(sub_test_list, limit_per_sub=100, limit_per_user=500, user_list=[]):
    sub_list = list(df_sub.sub_name[df_sub.sub_name.notnull()].unique())
    df_comment = pd.DataFrame()
    user_set = set(user_list)
    for index, sub in enumerate(sub_test_list):
        print('{}/{} Collecting user_info for subreddit {}. Time: {}.'.format(index, len(sub_test_list), sub, strftime("%H:%M:%S", gmtime())))
        
        df_user, user_set = user_info(sub, user_set, limit_per_sub)
        
        df_comment_s = user_sub_comment(df_user, sub_list, limit_per_user)
        df_user = None

        df_comment = pd.concat([df_comment, df_comment_s])
        df_comment_s = None
    
    return df_comment, user_set
    

In [304]:
sub_list = list(df_sub.sub_name[df_sub.sub_name.notnull()].unique())

In [305]:
sub_list.index('Psychonaut')

34

In [306]:
len(sub_list)

132

In [293]:
sub_list[35]

'Psychonaut'

In [280]:
df_comment0.head()

,user,comment,subreddit,score,time
0,Neotokyo199X,Nope.,GTAV,2,1.590249e+09
1,Neotokyo199X,That's because it is. Look at the gray pixels ...,apexlegends,3,1.589737e+09
2,Neotokyo199X,What the fuck is this,GTAV,2,1.589566e+09
3,Neotokyo199X,Except for driving games. Or really anything t...,apexlegends,2,1.589563e+09
4,Neotokyo199X,Another reason could be that the voice chat in...,apexlegends,2,1.589396e+09


In [279]:
len(df_comment0.user.unique())

1020

In [281]:
len(df_comment0.subreddit.unique())

126

In [324]:
df_comment0, user_set0 = get_comment(sub_list[:20])

0/20 Collecting user_info for subreddit GTAV. Time: 16:57:46.


100%|██████████████████████████████████████████████████████████████████████████████████| 71/71 [02:48<00:00,  2.37s/it]


1/20 Collecting user_info for subreddit Portal. Time: 17:00:37.


100%|██████████████████████████████████████████████████████████████████████████████████| 66/66 [03:00<00:00,  2.73s/it]


2/20 Collecting user_info for subreddit thewitcher3. Time: 17:03:39.


...

100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [03:24<00:00,  3.36s/it]


18/20 Collecting user_info for subreddit horizon. Time: 17:54:39.


100%|██████████████████████████████████████████████████████████████████████████████████| 66/66 [02:26<00:00,  2.21s/it]


19/20 Collecting user_info for subreddit uncharted. Time: 17:57:06.


100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [02:03<00:00,  2.06s/it]


In [335]:
df_comment1, user_set1 = get_comment(sub_list[20:80], user_list = list(user_set0))

0/60 Collecting user_info for subreddit civ. Time: 18:35:01.


100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [03:28<00:00,  3.20s/it]


1/60 Collecting user_info for subreddit thelastofus. Time: 18:38:31.


100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [01:58<00:00,  3.21s/it]


2/60 Collecting user_info for subreddit masseffect. Time: 18:40:31.


...

100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [02:18<00:00,  2.23s/it]


58/60 Collecting user_info for subreddit HiTMAN. Time: 21:22:35.


100%|██████████████████████████████████████████████████████████████████████████████████| 67/67 [03:13<00:00,  2.88s/it]


59/60 Collecting user_info for subreddit maxpayne. Time: 21:25:50.


100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [03:04<00:00,  2.63s/it]


In [338]:
df_comment2, user_set2 = get_comment(sub_list[80:], user_list = list(user_set1))

0/52 Collecting user_info for subreddit Persona5. Time: 21:42:46.


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [02:14<00:00,  2.69s/it]


1/52 Collecting user_info for subreddit celestegame. Time: 21:45:02.


100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [02:16<00:00,  2.69s/it]


2/52 Collecting user_info for subreddit Crysis. Time: 21:47:21.


...

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [01:51<00:00,  2.19s/it]


50/52 Collecting user_info for subreddit FinalFantasy. Time: 23:48:39.


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [02:45<00:00,  3.32s/it]


51/52 Collecting user_info for subreddit persona4golden. Time: 23:51:26.


100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [02:48<00:00,  2.19s/it]


In [337]:
sub_list[80]

'Persona5'

In [330]:
len(user_set0)

1151

In [332]:
len(df_comment0.user.unique())

532

In [326]:
df_comment0.to_csv('data\df_comment0.csv')

In [339]:
df_comment1.to_csv('data\df_comment1.csv')
df_comment2.to_csv('data\df_comment2.csv')

In [341]:
df_comment0.shape, df_comment1.shape, df_comment2.shape

((96520, 5), (377913, 5), (235657, 5))

In [342]:
len(user_set0), len(user_set1), len(user_set2)

(1151, 4746, 7670)

In [343]:
df_comment = pd.concat([df_comment0, df_comment1, df_comment2])

In [344]:
df_comment.shape

(710090, 5)

In [345]:
df_comment.to_csv('data\df_comment.csv')

In [347]:
df_comment.head()

,user,comment,subreddit,score,time
0,Neotokyo199X,Nope.,GTAV,2,1.590249e+09
1,Neotokyo199X,That's because it is. Look at the gray pixels ...,apexlegends,3,1.589737e+09
2,Neotokyo199X,What the fuck is this,GTAV,2,1.589566e+09
3,Neotokyo199X,Except for driving games. Or really anything t...,apexlegends,2,1.589563e+09
4,Neotokyo199X,Another reason could be that the voice chat in...,apexlegends,2,1.589396e+09


In [349]:
len(df_comment.user.unique()), len(df_comment.subreddit.unique())

(3731, 131)

In [352]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option("default_format", None)

In [353]:
df_user_sub = df_comment.groupby(['user', 'subreddit']).count()

In [ ]:
df_